# Calculate

Calculate any additional variables that will be compared between WSRA and SWIFT.

In [ ]:
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from sharedfunctions import read_stored_variable

## Setup

Read stored variables from `io.pynb`

In [ ]:
%run 'io.ipynb'
%run -i sharedfunctions.py

atomic_wsra = read_stored_variable('atomic_wsra')
atomic_swifts = read_stored_variable('atomic_swifts')

## SWIFT

Calculate bulk parameters from the SWIFT spectra.

In [ ]:
def spectral_moment(energy_density, frequency=None, n=0):
    """
    Compute 'nth' spectral moment

    Input:
        - energy, input array of energy densities ([n,1] arr OR [n,m] ndarr)
        - freq, input array of angular_frequencies ([n,1] arr OR [n,m] ndarr)
        - n, moment ([1,] int)

    Output:
        - mn, nth spectral moment ([1,] float)
            * if energy is empty or invalid, mn is assigned a NaN

    Example:

    Compute 4th spectral moment:
        m4 = spectral_moment(energy, freq, n=4)
    """
    if hasattr(energy_density, '__len__') and (not isinstance(energy_density, str)):
        fn = frequency ** n
        mn = np.trapz(energy_density * fn, x=frequency)  # axis=1
    else:
        mn = np.NaN
    return mn

### Energy-weighted mean direction

Compute the energy-weighted mean direction for the SWIFT buoys.  This metric should be more stable than the peak period.

In [ ]:
def energy_weighted_mean(
    X: np.ndarray,
    energy_density: np.ndarray,
    frequency: np.ndarray
) -> np.ndarray:
    """ Compute the energy-weighted mean of the input variable X.

    Args:
        X (np.ndarray): frequency-dependent variable of shape (f,n)
        energy_density (np.ndarray): spectral energy density of shape (f,n)
        frequency (np.ndarray): frequency vector of shape (f,n)

    Returns:
        np.ndarray: energy weighted mean of X with shape (n)
    """
    m0 = spectral_moment(energy_density, frequency, n=0)
    weighted_integral = np.trapz(y=energy_density*X, x=frequency)
    return weighted_integral / m0

def direction(a1: np.ndarray, b1: np.ndarray) -> np.ndarray:
    """Compute direction from directional moments a1 and b1.

    Args:
        a1 (np.ndarray): normalized spectral directional moment (+E)
        b1 (np.ndarray): normalized spectral directional moment (+N)

    Returns:
        np.ndarray: direction in meteorological convention
    """
    return (90 - np.rad2deg(np.arctan2(b1, a1))) % 360

In [ ]:
for swift_id, swift_ds in atomic_swifts.items():
    energy = swift_ds['energy'].values.T
    energy[np.isnan(energy)] = 0
    frequency = swift_ds['freq'].values
    a1_weighted = energy_weighted_mean(swift_ds['a1'].values.T, energy, frequency)
    b1_weighted = energy_weighted_mean(swift_ds['b1'].values.T, energy, frequency)
    swift_ds['mean_direction'] = ('time', direction(a1_weighted, b1_weighted))

### Mean square slope

Compute the spectral mean square slope for each SWIFT.

In [ ]:
def mean_square_slope(energy_density: np.ndarray, frequency: np.ndarray) -> np.ndarray:
    """Compute spectral mean square slope.

    Args:
        energy_density (np.ndarray): spectral energy density of shape (f,n)
        frequency (np.ndarray): frequency vector of shape (f,n)

    Returns:
        np.ndarray: mean square slope with shape (n,)
    """
    ACC_GRAV = 9.81
    fourth_moment = spectral_moment(energy_density, frequency=frequency, n=4)
    return (2*np.pi)**4 * fourth_moment / (ACC_GRAV**2)

In [ ]:
for swift_id in atomic_swifts.keys():
    energy = atomic_swifts[swift_id]['energy'].values.T
    energy[np.isnan(energy)] = 0
    frequency = atomic_swifts[swift_id]['freq'].values
    mss = mean_square_slope(energy_density=energy, frequency=frequency)
    atomic_swifts[swift_id]['mean_square_slope'] = ('time', mss)

### Day 

Create a `day` variable which represents the date rounded down to the nearest day.  This will be used to isolate ATOMIC missions.

In [ ]:
for swift_id in atomic_swifts.keys():
    atomic_swifts[swift_id]['day'] = ('time', atomic_swifts[swift_id].time.dt.floor("D").values)

In [ ]:
%%capture
%store atomic_swifts

## WSRA

### Timestamp

Create a `timestamp` variable which represents the time in numeric format.  This will be used for plotting and any interpolation.

In [ ]:
atomic_wsra['timestamp'] = ('time', pd.to_numeric(atomic_wsra['time'].values))

### Day and hour

Create `day` and `hour` variables which represent the date rounded down to the nearest day and datetime rounded down to the nearest hour.  This will be used to isolate ATOMIC missions.

In [ ]:
atomic_wsra['day'] = ('time', atomic_wsra.time.dt.floor("D").values)
atomic_wsra['hour'] = ('time', atomic_wsra.time.dt.floor("H").values)

### Mask

Create a trajectory mask and overwrite the original data.  Use the limits specified in Pincus et al. (2021).

In [ ]:
atomic_wsra.wsra.create_trajectory_mask(altitude_limits=(500, 4000),
                                        roll_limit=3)
atomic_wsra = atomic_wsra.wsra.mask()

### Colocate

Colocate WSRA and SWIFT observations to within 1 hour and 55 km (0.5 deg of lat).  The colocation is performed per day (per mission) and the matching indices are stored in a DataFrame for repeated use.

In [ ]:
def great_circle_pairwise(
    longitude_a: np.ndarray,
    latitude_a: np.ndarray,
    longitude_b: np.ndarray,
    latitude_b: np.ndarray,
    earth_radius: float = 6378.137,
    mod_bearing: bool = True
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computes the great circle distance (km) and true fore bearing (deg) between
    pairs of observations in input arrays `longitude_a` and `longitude_b` and
    `latitude_a` and `latitude_b`.

    For two longitude and latitude pairs, the great circle distance is the
    shortest distance between the two points along the Earth's surface. This
    distance is calculated using the Haversine formula. The instances in
    `longitude_a` and `latitude_a` are designated as point `a`; the instances
    in `longitude_b` and `latitude_b` then form point `b`. The true fore
    bearing is the bearing, measured from true north, of `b` as seen from `a`.

    Args:
        longitude_a (np.array): of shape (n,) in units of decimal degrees
        latitude (np.array): of shape (n,) in units of decimal degrees
        earth_radius (float, optional): earth's radius in units of km. Defaults to 6378.137 km (WGS-84)
        mod_bearing (bool, optional): return bearings modulo 360 deg. Defaults to True.

    Returns:
        Tuple[np.array, np.array]: great circle distances (in km) and true fore
        bearings between adjacent longitude and latitude pairs; shape (n,)
    """
    # Convert decimal degrees to radians
    longitude_a_rad, latitude_a_rad = map(np.radians, [longitude_a, latitude_a])
    longitude_b_rad, latitude_b_rad = map(np.radians, [longitude_b, latitude_b])

    # Difference longitude and latitude
    longitude_difference = longitude_b_rad - longitude_a_rad
    latitude_difference = latitude_b_rad - latitude_a_rad

    # Haversine formula
    a_1 = np.sin(latitude_difference / 2) ** 2
    a_2 = np.cos(latitude_a_rad)
    a_3 = np.cos(latitude_b_rad)
    a_4 = np.sin(longitude_difference / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a_1 + a_2 * a_3 * a_4))
    distance_km = earth_radius * c

    # True bearing
    bearing_num = np.cos(latitude_b_rad) * np.sin(-longitude_difference)
    bearing_den_1 = np.cos(latitude_a_rad) * np.sin(latitude_b_rad)
    bearing_den_2 = - np.sin(latitude_a_rad) * np.cos(latitude_b_rad) * np.cos(longitude_difference)
    bearing_deg = -np.degrees(np.arctan2(bearing_num, bearing_den_1 + bearing_den_2))

    if mod_bearing:
        bearing_deg = bearing_deg % 360

    return distance_km, bearing_deg


def colocate_with_path(
    wsra_ds: xr.Dataset,
    path_ds: xr.Dataset,
    path_vars: Tuple,
    wsra_vars: Tuple = ('time', 'latitude', 'longitude'),
    temporal_tolerance: np.timedelta64 = np.timedelta64(30, 'm'),
    spatial_tolerance: float = 1.0,  # km
) -> Tuple[np.ndarray, np.ndarray,  np.ndarray,  np.ndarray]:
    """
    Find matching WSRA observations with data along another path (i.e. a
    drifting buoy) based on temporal and spatial tolerances.

    Note:
        `path_vars` and `wsra_vars` are tuples specifying the names of the
        coordinates and fields to interpolate. The names must be ordered as:
        time, latitude, longitude.

        For instance, if the path dataset coordinates are labeled as 'time',
        'lat', and 'lon', then `path_vars` should be:
        >>> path_vars = ('time', 'lat', 'lon')

        `wsra_vars` defaults to the standard dataset names, though these should
        be provided if the defaults have been modified.

    Args:
        wsra_ds (xr.Dataset): WSRA observations
        path_ds (xr.Dataset): path data
        temporal_tolerance (np.timedelta64, optional): max allowable time delta
            between WSRA and path times. Defaults to np.timedelta64(30, 'm').
        spatial_tolerance (float, optional): max allowable distance
            between WSRA and path times. Defaults to 1.0 km.

    Returns:
        np.ndarray: matching WSRA indices
        np.ndarray: matching path indices
        np.ndarray: great circle distance at each match
        np.ndarray: time difference at each match
    """

    wsra_time = wsra_ds[wsra_vars[0]].values
    wsra_latitude = wsra_ds[wsra_vars[1]].values
    wsra_longitude = wsra_ds[wsra_vars[2]].values

    path_time = path_ds[path_vars[0]].values
    path_latitude = path_ds[path_vars[1]].values
    path_longitude = path_ds[path_vars[2]].values

    # Get the indices where the WSRA times fit within the path times
    t_sort_indices = np.searchsorted(path_time, wsra_time)
    t_sort_indices[t_sort_indices >= len(path_time)] = len(path_time)-1

    time_difference = np.abs(wsra_time - path_time[t_sort_indices])
    in_time = time_difference < temporal_tolerance

    # Great circle distance along the path
    distance, bearing = great_circle_pairwise(
        longitude_a=wsra_longitude,
        latitude_a=wsra_latitude,
        longitude_b=path_longitude[t_sort_indices],
        latitude_b=path_latitude[t_sort_indices]
    )

    in_range = distance < spatial_tolerance

    # Matches are where time and distance constraints are satisfied.
    matching_boolean = np.logical_and(in_time, in_range)

    # Get matching WSRA and path indices and the distance and time difference
    # at each match.
    wsra_indices = np.where(matching_boolean)[0]
    path_indices = t_sort_indices[matching_boolean]
    distances = distance[matching_boolean]
    time_differences = time_difference[matching_boolean]

    return wsra_indices, path_indices, distances, time_differences

For each mission, get the matching indices and collect them in the DataFrame `matches_df`.

In [ ]:
mission_dates = np.unique(atomic_wsra['day'].dropna(dim='time'))

matches_dict = []
for date in mission_dates:
    wsra_in_mission = atomic_wsra.where(atomic_wsra['day'] == date)  #, drop=True)

    for swift_id in atomic_swifts.keys():
        match_dict = {}
        swift_in_mission = atomic_swifts[swift_id].where(atomic_swifts[swift_id]['day'] == date)

        wsra_indices, swift_indices, distance, time_difference \
            = colocate_with_path(
                wsra_ds = wsra_in_mission,
                path_ds = swift_in_mission,
                path_vars = ('time', 'lat', 'lon', ''),
                wsra_vars = ('time', 'latitude', 'longitude'),
                temporal_tolerance = np.timedelta64(60, 'm'),
                spatial_tolerance = 55,  #km
        )
        match_dict['date'] = date
        match_dict['swift_id'] = swift_id
        match_dict['wsra_indices'] = wsra_indices
        match_dict['swift_indices'] = swift_indices
        match_dict['distance'] = distance
        match_dict['time_difference'] = time_difference.astype('timedelta64[s]')
        matches_dict.append(match_dict)

matches_df = (pd.DataFrame(matches_dict)
    .dropna()
    .set_index(['date', 'swift_id'])
)


In [ ]:
%%capture
%store atomic_wsra
%store matches_df